V3 base gbdt

V4 base lightgbm

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
%matplotlib inline


Use first 10**5 rows of train dataset for data exploring. Using more efficient datatypes as shown in introduction notebook.

Exploring questions stats

In [ ]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                         usecols=['row_id', 'user_id', 'answered_correctly', 'content_id', 'prior_question_had_explanation', 'prior_question_elapsed_time'],
                         dtype={'row_id': 'int64',  'user_id': 'int32', 'content_id': 'int16', 'answered_correctly': 'int8', 'prior_question_had_explanation': 'boolean', 'prior_question_elapsed_time':'float32'}
                         )

train_df.shape

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
user_df_inf = train_df[train_df.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['mean', 'median', 'std', 'skew', 'count']}).reset_index()
user_df_inf.columns = ['user_id',
                   'mean_user_acc',
    'median_user_acc',
    'std_user_acc',
    'skew_user_acc',
    'number_of_answered_q']
user_df_inf

In [ ]:
content_df = train_df[train_df.answered_correctly != -1].groupby('content_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
content_df.columns = ['content_id', 'content_questions', 'content_mean']
content_df.head()

In [ ]:
user_lect = train_df.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['lecture', 'wrong', 'right']
user_lect['lecture'] = user_lect['lecture'].fillna(0)
user_lect = user_lect.astype('Int64')
user_lect['watches_lecture'] = np.where(user_lect.lecture > 0, 1, 0)
user_lect = user_lect.reset_index()
user_lect = user_lect[['user_id', 'watches_lecture']]

In [ ]:
user_lect

In [ ]:
user_df = user_df_inf.merge(user_lect, on = "user_id", how = "left")
del user_lect,train_df,user_df_inf
user_df.head()

In [ ]:
train_df = pd.read_pickle("../input/riidcv/cv1_train.pickle")
validation_df = pd.read_pickle("../input/riidcv/cv1_valid.pickle")

In [ ]:
train_df.head()

In [ ]:
import gc
label_enc = LabelEncoder()
mean_prior = train_df.prior_question_elapsed_time.astype("float64").mean()
train_df= train_df.merge(user_df, on = "user_id", how = "left")
train_df = train_df.merge(content_df, on = "content_id", how = "left")


In [ ]:
train_df.head()

In [ ]:
train_df['content_questions'].fillna(0, inplace = True)
train_df['content_mean'].fillna(0.5, inplace = True)
train_df['watches_lecture'].fillna(0, inplace = True)
train_df['number_of_answered_q'].fillna(0, inplace = True)
train_df['mean_user_acc'].fillna(0.5, inplace = True)
train_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train_df['prior_question_had_explanation'].fillna(False, inplace = True)
train_df['prior_question_had_explanation'] = label_enc.fit_transform(train_df['prior_question_had_explanation'])
train_df[['content_questions', 'number_of_answered_q']] = train_df[['content_questions', 'number_of_answered_q']].astype(int)
train_df.sample(5)

In [ ]:
validation_df = validation_df.merge(user_df, on = "user_id", how = "left")
validation_df = validation_df.merge(content_df, on = "content_id", how = "left")


In [ ]:
validation_df['content_questions'].fillna(0, inplace = True)
validation_df['content_mean'].fillna(0.5, inplace = True)
validation_df['watches_lecture'].fillna(0, inplace = True)
validation_df['number_of_answered_q'].fillna(0, inplace = True)
validation_df['mean_user_acc'].fillna(0.5, inplace = True)
validation_df['prior_question_had_explanation'].fillna(False, inplace = True)
validation_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
validation_df['prior_question_had_explanation'] = label_enc.fit_transform(validation_df['prior_question_had_explanation'])
validation_df[['content_questions', 'number_of_answered_q']] = validation_df[['content_questions', 'number_of_answered_q']].astype(int)
validation_df.sample(5)

In [ ]:
train_df.shape

In [ ]:
features = ['number_of_answered_q', 'content_questions', 'content_mean', 
            'prior_question_had_explanation', 'prior_question_elapsed_time', 'watches_lecture','mean_user_acc','std_user_acc','skew_user_acc','number_of_answered_q']


train = train_df

y_train = train['answered_correctly']
train = train[features]

y_val = validation_df['answered_correctly']
validation = validation_df[features]

In [ ]:
lgb_train = lgb.Dataset(train, y_train)
lgb_eval = lgb.Dataset(validation, y_val)
del train, y_train, validation, y_val
gc.collect()

In [ ]:
# params = {'objective': 'binary',
#           'metric': 'auc',
#           'seed': 42,
#           'learning_rate': 0.1, 
#           "boosting_type": "gbdt",
#          }
# model = lgb.train(
#     params, lgb_train,
#     valid_sets=[lgb_train, lgb_eval],
#     verbose_eval=1,
#     num_boost_round=10000,
#     early_stopping_rounds=8
# )
model = lgb.train(
                    {'objective': 'binary',#,#,
                        'metric': 'auc',
                   #  'num_iterations':1,
                    },#50

                    lgb_train,
                    valid_sets=[lgb_train, lgb_eval],
                    verbose_eval=10,
                    num_boost_round=10000,
                    early_stopping_rounds=10
                )
_ = lgb.plot_importance(model)

In [ ]:
# predictions = pd.DataFrame(model.predict(validation),index=validation.index)

# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_val,predictions[0])

In [ ]:
# params = {
#     'objective': 'binary',
#     'seed': 42,
#     'metric': 'auc',
#     'learning_rate': 0.05,
#     'max_bin': 1000,
#     'num_leaves': 80,
#     'num_iterations' : 1
# }
# model = lgb.train(
#     params,
#     lgb_train,
#     valid_sets=[lgb_train,lgb_eval],
#     verbose_eval=5,
#     num_boost_round=100,
#     early_stopping_rounds=10
# )

In [ ]:
import riiideducation
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(user_df, on = "user_id", how = "left")
    test_df = test_df.merge(content_df, on = "content_id", how = "left")
    test_df['content_questions'].fillna(0, inplace = True)
    test_df['content_mean'].fillna(0.5, inplace = True)
    test_df['watches_lecture'].fillna(0, inplace = True)
    test_df['number_of_answered_q'].fillna(0, inplace = True)
    test_df['mean_user_acc'].fillna(0.5, inplace = True)
    test_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace = True)
    test_df['prior_question_had_explanation'] = label_enc.fit_transform(test_df['prior_question_had_explanation'])
    test_df[['content_questions', 'number_of_answered_q']] = test_df[['content_questions', 'number_of_answered_q']].astype(int)
    test_df['answered_correctly'] =  model.predict(test_df[features])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']]) 